In [76]:
import nltk
from nltk.corpus import wordnet as wn
import csv
import string

import nltk
from nltk.corpus import stopwords

countlines = 0 

allwords = []
with open("train.tsv", encoding="utf8") as tsvfile:    #open training set
    lines = csv.reader(tsvfile, delimiter="\t")        #convert file to lines
    for line in lines:
        statement = line[2]                                #get statement from each line 
        lostrings = statement.split(" ")                   #convert string to list of strings
        new_lostrings = []
        countlines += 1 
        for word in lostrings:
            word = nltk.WordNetLemmatizer().lemmatize(
                word.translate(str.maketrans('', '', string.punctuation)).lower()) # remove punctuation & lemmatize
            if word not in stopwords.words('english') and not word.isdigit():
                new_lostrings.append(word)
        allwords.extend(new_lostrings)

vocab = []                          #initialize a list for all the distint words in the trainset
for word in allwords:
    if word in vocab:               #do not add word if word is already in vocabulary 
        continue 
    else:
        vocab.append(word)


print("Number of words in trainset:", len(allwords))
print("Number of distinct words in trainset:", len(vocab))
print(allwords[:25])
print(vocab[:25])
print(type(vocab))
print(countlines)
print(len(allwords))

Number of words in trainset: 110580
Number of distinct words in trainset: 11057
['say', 'annies', 'list', 'political', 'group', 'support', 'thirdtrimester', 'abortion', 'demand', 'decline', 'coal', 'start', 'started', 'natural', 'gas', 'took', 'started', 'begin', 'president', 'george', 'w', 'bush', 'administration', 'hillary', 'clinton']
['say', 'annies', 'list', 'political', 'group', 'support', 'thirdtrimester', 'abortion', 'demand', 'decline', 'coal', 'start', 'started', 'natural', 'gas', 'took', 'begin', 'president', 'george', 'w', 'bush', 'administration', 'hillary', 'clinton', 'agrees']
<class 'list'>
10240
110580


In [78]:
from collections import Counter
list1 = allwords
counts = Counter(list1)

for word in vocab: 
    if counts[word] == 1:
        vocab.remove(word)

print(len(vocab))

6515


In [79]:
#make a list containing all the statements for the train data 

statements = []
labels = []

with open("train.tsv", encoding="utf8") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        label = line[1]
        aline = line[2]
        bline = aline.split(" ")
        for word in bline:
            if word not in vocab:
                bline.remove(word)
        cline = []
        for word in bline:
            word = nltk.WordNetLemmatizer().lemmatize(
                word.translate(str.maketrans('', '', string.punctuation)).lower()) # remove punctuation & lemmatize
            if word not in stopwords.words('english') and not word.isdigit():
                cline.append(word)
        labels.append(label)
        statements.append(cline)

print("First 3 statements:", statements[:3])
print("First 3 labels:", labels[:3])

First 3 statements: [['list', 'political', 'group', 'thirdtrimester'], ['decline', 'coal', 'started', 'natural', 'gas', 'took', 'started', 'begin', 'president', 'w', 'administration'], ['clinton', 'agrees', 'john', 'voting', 'give', 'bush', 'benefit', 'doubt', 'iran']]
First 3 labels: ['false', 'half-true', 'mostly-true']


In [80]:
import numpy as np 
rows = countlines
columns = len(vocab) 
matrix = np.zeros((rows, columns))

In [95]:
#Create matrix representation, X_matrix 

counts1 = 0 
for statement in statements: 
    counts2 = 0
    for word in vocab:
        if word in statement:
            count = statement.count(word)
            matrix[counts1, counts2] = count
        counts2 += 1 
    counts1 += 1 

print(matrix[0])
print(len(matrix[0]))

[0. 1. 1. ... 0. 0. 0.]
6515


In [81]:
#Create y vector
labelsdic ={"false":0, "barely-true":1,"half-true":2,"mostly-true":3,"true":4, "pants-fire":5}

size = len(statements)
y_vector = [None] * size

counter = 0
for label in labels:
    y_vector[counter] = labelsdic[label]
    counter += 1

In [82]:
from sklearn.linear_model import LogisticRegression

In [96]:
lr = LogisticRegression(solver='lbfgs',multi_class='multinomial')
lr.fit(matrix, y_vector)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [97]:
#make list of statements for test data set 
import string 
import nltk
from nltk.corpus import wordnet as wn
import csv

teststatements = []
testlabels = []
test_countlines = 0

with open("test.tsv", encoding="utf8") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t")
    for line in tsvreader:
        label = line[1]
        aline = line[2]
        bline = aline.split(" ")
        for word in bline:
            if word not in vocab:
                bline.remove(word)
        cline = []
        test_countlines += 1 
        for word in bline:
            word = nltk.WordNetLemmatizer().lemmatize(
                word.translate(str.maketrans('', '', string.punctuation)).lower()) # remove punctuation & lemmatize
            if word not in stopwords.words('english') and not word.isdigit():
                cline.append(word)
        testlabels.append(label)
        teststatements.append(cline)

print("First 3 statements:", teststatements[:3])
print("First 3 labels:", testlabels[:3])
print(test_countlines)

First 3 statements: [['wall', 'border', 'take', 'literally'], ['pace', 'double', 'number', 'layoff', 'year'], ['john', 'ha', 'done', 'nothing', 'help', 'vet']]
First 3 labels: ['true', 'false', 'false']
1267


In [93]:
import numpy as np 
rows = test_countlines
columns = len(vocab)
print(len(vocab))
testmatrix = np.zeros((rows, columns))

6515


In [94]:
#Create matrix representation for test dataset, X_matrix_test 

counts1 = 0 
for statement in teststatements: 
    counts2 = 0
    for word in vocab:
        if word in statement:
            count = statement.count(word)
            testmatrix[counts1, counts2] = count
        counts2 += 1 
    counts1 += 1 

print(testmatrix[0])
print(len(testmatrix[0]))

[0. 0. 0. ... 0. 0. 0.]
6515


In [88]:
#Create test y vector
labelsdic ={"false":0, "barely-true":1,"half-true":2,"mostly-true":3,"true":4, "pants-fire":5}

size = len(teststatements)
test_y_vector = [None] * size

counter = 0
for label in testlabels:
    test_y_vector[counter] = labelsdic[label]
    counter += 1

In [98]:
test_x = testmatrix 
test_y = test_y_vector

In [99]:
lr.score(test_x, test_y)

0.2296764009471192